In [ ]:
# libraries

# Python 3.9.12

import pandas as pd                    # pandas==1.5.3

import numpy as np                     # numpy==1.23.5        

from scipy.optimize import minimize    # scipy==1.10.0 

from numba import jit                  # numba==0.56.4


import plotly                          # plotly==5.13.0
import plotly.express as px

## Model

In [ ]:
class PoissonPriceModel:
    
    '''
    class PoissonPriceModel
    '''
    
    def __init__(self):
        
        # initial params
        self.params = None
    
    
    # the decorator from numba is for 3X speed in fitting
    @staticmethod
    @jit()  
    def loglike(price: float, count: int, alfa: float, beta: float) -> float:

        '''
        Log-Likelihood from demand exponential function:

        demand = np.exp(alfa*price + beta)

        Args:
        price : float, product price
        count : int, product quantity sold in that price
        alfa : float, first fit parameter, price parameter
        beta : float, second fit parameter, intercept parameter


        Return:
        log-likelihood : float 
        '''

        demand = np.exp(alfa*price + beta)

        return -demand + count*np.log(demand)
    
    
    
    def fit(self, X: np.array) -> np.array:
    
        '''
        Fit model function. 

        Args:
        X : np.array with shape (N, 2), first column is price and the second is count_

        Return:
        params : np.array with two float, alfa and beta
        '''


        # array of alfa and beta initial values 
        initial_values = np.array([-1., 0.]) # alfa, beta               


        def minus_loglike(params: np.array) -> float:  # Log-Likelihood, function to be minimize

            '''
            Args:
            params : np.array with two float, alfa and beta

            Return:
            minu-log-likelihood : float, minus for maximization
            '''


            log_like = map(lambda elem: PoissonPriceModel().loglike(elem[0],     # price from data
                                                                    elem[1],     # count from data
                                                                    params[0],   # alfa parameter
                                                                    params[1]),  # beta parameter
                                               X)



            return -sum(log_like) 


        output=minimize(minus_loglike, 
                        initial_values, 
                        options={'disp':False, 'maxiter':100}, 
                        method='L-BFGS-B') 
        
        self.params = output.x   # alfa, beta
    
    
    
    def predict(self) -> list:
    
        '''
        Function to predict demand & profit functions from parameters.

        Args:
        params : np.array with two float, alfa and beta

        Return:
        demand & profit functions : list with two np.array with 1000 ordered values each
        '''

        x = np.linspace(1, 100, 1000)

        demand = np.exp(self.params[0]*x + self.params[1])

        profit = x * demand * 100

        return [demand, profit]

## Data

Sample data group by date and price with count as aggregation function. Data for model feeding.

In [ ]:
data = pd.read_parquet('data.parquet')

data.head()

In [ ]:
data.info(memory_usage='deep')

In [ ]:
data.describe(datetime_is_numeric=True)

## Fit and predict

In [ ]:
X = data[['price', 'count_']]

X.head()

In [ ]:
X = X.values

X[0]   # price, count

In [ ]:
X.shape

In [ ]:
## Poisson model

model = PoissonPriceModel()

In [ ]:
%%time

model.fit(X)

In [ ]:
prediction = model.predict()

## Plots

In [ ]:
df_plot=pd.DataFrame()

df_plot['rate'] = np.linspace(1, 100, 1000)

df_plot['demand'] = prediction[0]

df_plot['profit'] = prediction[1]

df_plot.head()

In [ ]:
# demand plot

px.line(df_plot, x='rate', y='demand')

In [ ]:
# profit plot

px.line(df_plot, x='rate', y='profit')